In [1]:
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from typing import Sequence
from tqdm import tqdm
import os

C:\Users\NightMare\anaconda3\envs\envGPU\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\NightMare\anaconda3\envs\envGPU\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\Users\NightMare\anaconda3\envs\envGPU\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
from argostranslate import package
package.install_from_path("C:/Users/NightMare/Desktop/neurofeed_back/translate-en_ru-1_9.argosmodel")

In [23]:
from argostranslate import translate

def get_argos_model(source, target):
    lang = f'{source} -> {target}'
    source_lang = [model for model in translate.get_installed_languages() if lang in map(repr, model.translations_from)]
    target_lang = [model for model in translate.get_installed_languages() if lang in map(repr, model.translations_to)]
    
    return source_lang[0].get_translation(target_lang[0])

argos_en_ru = get_argos_model('English', 'Russian')

In [24]:
argos_en_ru.translate('hello')
# "I think it's a disease, a real, complete disease."

'привет'

In [12]:
path_to_datasets = "C:/Users/NightMare/Desktop/neurofeed_back/MOOC_Reviews1.csv"

In [13]:
# Используйте метод read_csv() из библиотеки Pandas для чтения CSV файла
df = pd.read_csv(path_to_datasets, index_col=0)

In [14]:
df

,Review,Review score,Lable
Id,,,
0,good and interesting,5,Positive
1,"This class is very helpful to me. Currently, I...",5,Positive
2,like!Prof and TAs are helpful and the discussi...,5,Positive
3,Easy to follow and includes a lot basic and im...,5,Positive
4,Really nice teacher!I could got the point eazl...,4,Positive
...,...,...,...
106939,I want to give it 3 1/2 starts - sometimes the...,3,Neutral
106955,I loved the non-UK / American basis for the wr...,3,Neutral
106968,"It is perfect for beginners, but if you're an ...",3,Neutral


In [15]:
# Удаляем столбец 'Numbers'
df = df.drop('Review score', axis=1)

# Удаление строк с меткой "Neutral"
df = df[df['Lable'] != 'Neutral']

In [16]:
df

,Review,Lable
Id,,
0,good and interesting,Positive
1,"This class is very helpful to me. Currently, I...",Positive
2,like!Prof and TAs are helpful and the discussi...,Positive
3,Easy to follow and includes a lot basic and im...,Positive
4,Really nice teacher!I could got the point eazl...,Positive
...,...,...
106897,I wanted this course to focus specifically on ...,Negative
106898,Information presented was too basic.,Negative
106928,The course didn't feel as though it was aimed ...,Negative


In [18]:
# Переименование столбца "labels" в "label"
df.rename(columns={"Lable": "label"}, inplace=True)
df.rename(columns={"Review": "text"}, inplace=True)

In [19]:
import re

# Функция для удаления знаков препинания из текста
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

# Удаление знаков препинания из текста
df['text'] = df['text'].apply(remove_punctuation)

# Приведение текста к нижнему регистру
df['text'] = df['text'].str.lower()

In [20]:
df

,text,label
Id,,
0,good and interesting,Positive
1,this class is very helpful to me currently im ...,Positive
2,likeprof and tas are helpful and the discussio...,Positive
3,easy to follow and includes a lot basic and im...,Positive
4,really nice teacheri could got the point eazli...,Positive
...,...,...
106897,i wanted this course to focus specifically on ...,Negative
106898,information presented was too basic,Negative
106928,the course didnt feel as though it was aimed a...,Negative


In [21]:
# Заменяем значения эмоций согласно вашему условию
replacement_dict = {
    'Positive': 1,
    'Negative': 0
}

# Применяем замену значений в столбце 'Emotion'
df['label'] = df['label'].replace(replacement_dict)

In [22]:
df

,text,label
Id,,
0,good and interesting,1
1,this class is very helpful to me currently im ...,1
2,likeprof and tas are helpful and the discussio...,1
3,easy to follow and includes a lot basic and im...,1
4,really nice teacheri could got the point eazli...,1
...,...,...
106897,i wanted this course to focus specifically on ...,0
106898,information presented was too basic,0
106928,the course didnt feel as though it was aimed a...,0


In [26]:
# Функция для перевода текста
def translate_text(text):
    return argos_en_ru.translate(text)

# Применение функции перевода с использованием потоков
with ThreadPoolExecutor() as executor:
    tqdm.pandas()
    df['text'] = list(tqdm(executor.map(translate_text, df['text']), total=len(df)))

100%|██████████| 10116/10116 [2:22:53<00:00,  1.18it/s] 


In [27]:
df.to_csv("MOOC_Reviews1_ru.csv", index=False)

In [28]:
df

,text,label
Id,,
0,Хорошая и интересная,1
1,Этот класс очень полезен для меня в настоящее ...,1
2,"likeprof и tas полезны, и дискуссия среди студ...",1
3,легко следовать и включает в себя много основн...,1
4,действительно хороший учитель мог бы получить ...,1
...,...,...
106897,"Я хотел, чтобы этот курс был сосредоточен конк...",0
106898,Представленная информация слишком проста,0
106928,"Курс не чувствовал, как будто он был направлен...",0


In [37]:
path_to_datasets = "C:/Users/NightMare/Desktop/neurofeed_back/harvard_reviews.csv"

In [38]:
# Используйте метод read_csv() из библиотеки Pandas для чтения CSV файла
df = pd.read_csv(path_to_datasets, index_col=0)

In [39]:
df

,published_platform,rating,type,helpful_votes,title,text
published_date,,,,,,
2023-12-28T08:02:14-05:00,Mobile,5,review,0,Best classes and good environment,Good thanks for everything good work group 👍 h...
2023-12-12T00:38:26-05:00,Desktop,4,review,0,Harvard University,Harvard University was founded in 1636 and is ...
2023-12-10T08:21:35-05:00,Mobile,3,review,0,Walk around campus,We did a walk around most of the Harvard Campu...
2023-11-07T21:20:21-05:00,Desktop,4,review,0,Interesting Harvard University,We had a walk through the university grounds w...
2023-10-02T17:40:42-04:00,Desktop,5,review,0,Lovely university campus has many historic and...,Finally made it to Harvard!! This iconic univ...
...,...,...,...,...,...,...
2010-10-29T05:47:57-04:00,Desktop,3,review,0,amerikanischer campus fuer touristen,wenn man in boston ist hat manmeistens zeit au...
2010-08-23T06:08:51-04:00,Desktop,3,review,0,Weniger als erwartet,Von der Führung der Harvard University erhofft...
2010-08-23T00:06:41-04:00,Desktop,5,review,0,bellissima università,"da vedere questa città, studio sede di una del..."


In [41]:
import re

# Функция для определения языка текста
def detect_language(text):
    # Паттерн для поиска символов не из английского алфавита
    pattern = re.compile(r'[^\x00-\x7F]+')
    if pattern.search(text):
        return False # Если найдены символы других языков, возвращаем False
    else:
        return True # Если только английские символы, возвращаем True

# Применение функции detect_language к столбцам 'title' и 'text' для удаления строк с текстом на других языках
df = df[df['title'].apply(detect_language)]
df = df[df['text'].apply(detect_language)]

In [42]:
df

,published_platform,rating,type,helpful_votes,title,text
published_date,,,,,,
2023-12-12T00:38:26-05:00,Desktop,4,review,0,Harvard University,Harvard University was founded in 1636 and is ...
2023-12-10T08:21:35-05:00,Mobile,3,review,0,Walk around campus,We did a walk around most of the Harvard Campu...
2023-11-07T21:20:21-05:00,Desktop,4,review,0,Interesting Harvard University,We had a walk through the university grounds w...
2023-10-02T17:40:42-04:00,Desktop,5,review,0,Lovely university campus has many historic and...,Finally made it to Harvard!! This iconic univ...
2023-09-29T08:44:17-04:00,Desktop,4,review,0,Interesting walk around campus grounds & river,Very interesting to wander around the grounds ...
...,...,...,...,...,...,...
2012-08-20T19:07:16-04:00,Desktop,4,review,0,Molto suggestiva,"Anche qui, come al mit, vale la pena andarci p..."
2012-03-26T07:36:54-04:00,Desktop,5,review,1,Como en las peliculas,Excelente para una buena caminata. Se reviven ...
2012-02-14T19:06:35-05:00,Desktop,4,review,0,Tradicion y Elegancia,Paseo por las elegantes instalaciones de la pr...


In [43]:
# Замена значений в столбце "rating" в соответствии с условиями
df['rating'] = df['rating'].apply(lambda x: 1 if x == 5 else 0)

In [44]:
df

,published_platform,rating,type,helpful_votes,title,text
published_date,,,,,,
2023-12-12T00:38:26-05:00,Desktop,0,review,0,Harvard University,Harvard University was founded in 1636 and is ...
2023-12-10T08:21:35-05:00,Mobile,0,review,0,Walk around campus,We did a walk around most of the Harvard Campu...
2023-11-07T21:20:21-05:00,Desktop,0,review,0,Interesting Harvard University,We had a walk through the university grounds w...
2023-10-02T17:40:42-04:00,Desktop,1,review,0,Lovely university campus has many historic and...,Finally made it to Harvard!! This iconic univ...
2023-09-29T08:44:17-04:00,Desktop,0,review,0,Interesting walk around campus grounds & river,Very interesting to wander around the grounds ...
...,...,...,...,...,...,...
2012-08-20T19:07:16-04:00,Desktop,0,review,0,Molto suggestiva,"Anche qui, come al mit, vale la pena andarci p..."
2012-03-26T07:36:54-04:00,Desktop,1,review,1,Como en las peliculas,Excelente para una buena caminata. Se reviven ...
2012-02-14T19:06:35-05:00,Desktop,0,review,0,Tradicion y Elegancia,Paseo por las elegantes instalaciones de la pr...


In [45]:
import re

# Функция для удаления знаков препинания из текста
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

# Удаление знаков препинания из текста
df['text'] = df['text'].apply(remove_punctuation)

# Приведение текста к нижнему регистру
df['text'] = df['text'].str.lower()

In [46]:
df

,published_platform,rating,type,helpful_votes,title,text
published_date,,,,,,
2023-12-12T00:38:26-05:00,Desktop,0,review,0,Harvard University,harvard university was founded in 1636 and is ...
2023-12-10T08:21:35-05:00,Mobile,0,review,0,Walk around campus,we did a walk around most of the harvard campu...
2023-11-07T21:20:21-05:00,Desktop,0,review,0,Interesting Harvard University,we had a walk through the university grounds w...
2023-10-02T17:40:42-04:00,Desktop,1,review,0,Lovely university campus has many historic and...,finally made it to harvard this iconic univer...
2023-09-29T08:44:17-04:00,Desktop,0,review,0,Interesting walk around campus grounds & river,very interesting to wander around the grounds ...
...,...,...,...,...,...,...
2012-08-20T19:07:16-04:00,Desktop,0,review,0,Molto suggestiva,anche qui come al mit vale la pena andarci per...
2012-03-26T07:36:54-04:00,Desktop,1,review,1,Como en las peliculas,excelente para una buena caminata se reviven e...
2012-02-14T19:06:35-05:00,Desktop,0,review,0,Tradicion y Elegancia,paseo por las elegantes instalaciones de la pr...


In [47]:
# Удаляем столбец 'Numbers'
df = df.drop(['published_platform', 'type', 'helpful_votes', 'title'], axis=1)

In [48]:
df

,rating,text
published_date,,
2023-12-12T00:38:26-05:00,0,harvard university was founded in 1636 and is ...
2023-12-10T08:21:35-05:00,0,we did a walk around most of the harvard campu...
2023-11-07T21:20:21-05:00,0,we had a walk through the university grounds w...
2023-10-02T17:40:42-04:00,1,finally made it to harvard this iconic univer...
2023-09-29T08:44:17-04:00,0,very interesting to wander around the grounds ...
...,...,...
2012-08-20T19:07:16-04:00,0,anche qui come al mit vale la pena andarci per...
2012-03-26T07:36:54-04:00,1,excelente para una buena caminata se reviven e...
2012-02-14T19:06:35-05:00,0,paseo por las elegantes instalaciones de la pr...


In [49]:
# Переименование столбца "labels" в "label"
df.rename(columns={"rating": "label"}, inplace=True)

In [50]:
df

,label,text
published_date,,
2023-12-12T00:38:26-05:00,0,harvard university was founded in 1636 and is ...
2023-12-10T08:21:35-05:00,0,we did a walk around most of the harvard campu...
2023-11-07T21:20:21-05:00,0,we had a walk through the university grounds w...
2023-10-02T17:40:42-04:00,1,finally made it to harvard this iconic univer...
2023-09-29T08:44:17-04:00,0,very interesting to wander around the grounds ...
...,...,...
2012-08-20T19:07:16-04:00,0,anche qui come al mit vale la pena andarci per...
2012-03-26T07:36:54-04:00,1,excelente para una buena caminata se reviven e...
2012-02-14T19:06:35-05:00,0,paseo por las elegantes instalaciones de la pr...


In [51]:
# Сброс индексации
df.reset_index(drop=True, inplace=True)

# Перемещение столбца "label" правее столбца "text"
df = df[['text', 'label']]

In [52]:
df

,text,label
0,harvard university was founded in 1636 and is ...,0
1,we did a walk around most of the harvard campu...,0
2,we had a walk through the university grounds w...,0
3,finally made it to harvard this iconic univer...,1
4,very interesting to wander around the grounds ...,0
...,...,...
1952,anche qui come al mit vale la pena andarci per...,0
1953,excelente para una buena caminata se reviven e...,1
1954,paseo por las elegantes instalaciones de la pr...,0
1955,agradable y precioso merece la pena un paseo p...,1


In [53]:
# Функция для перевода текста
def translate_text(text):
    return argos_en_ru.translate(text)

# Применение функции перевода с использованием потоков
with ThreadPoolExecutor() as executor:
    tqdm.pandas()
    df['text'] = list(tqdm(executor.map(translate_text, df['text']), total=len(df)))

100%|██████████| 1957/1957 [41:18<00:00,  1.27s/it] 


In [55]:
df

,text,label
0,Гарвардский университет был основан в 1636 год...,0
1,Мы прогулялись по большей части кампуса Гарвар...,0
2,У нас была прогулка по университетской террито...,0
3,"Наконец, сделали это в Гарварде этот знаковый ...",1
4,Очень интересно побродить по территории кампус...,0
...,...,...
1952,ache qui come al mit vale la pena andarci per ...,0
1953,excelente para una buena caminata se reviven e...,1
1954,paseo por las elegantes instalaciones de la pr...,0
1955,agradable y precioso merece la pena un paseo p...,1


In [56]:
import re

# Функция для определения языка текста
def detect_russian(text):
    # Паттерн для поиска русских букв
    pattern = re.compile(r'[а-яА-ЯёЁ]+')
    if pattern.search(text):
        return True # Если найдены русские буквы, возвращаем True
    else:
        return False # Если только другие символы, возвращаем False

# Применение функции detect_russian к столбцу 'text' для удаления строк с текстом на других языках
df = df[df['text'].apply(detect_russian)]

# Сброс индексации
df.reset_index(drop=True, inplace=True)

In [57]:
df

,text,label
0,Гарвардский университет был основан в 1636 год...,0
1,Мы прогулялись по большей части кампуса Гарвар...,0
2,У нас была прогулка по университетской террито...,0
3,"Наконец, сделали это в Гарварде этот знаковый ...",1
4,Очень интересно побродить по территории кампус...,0
...,...,...
1819,LOPEND Over Het Groene gras van de Fantastisch...,1
1820,"Университетский городок Муи умирает, op het ha...",1
1821,piacevole ed interessante se prestate attenzio...,0
1822,Исполнитель: Suas Folhas Caindo Muito Legal\ne...,1


In [58]:
df.to_csv("harvard_reviews_ru.csv", index=False)

In [80]:
path_to_datasets = "C:/Users/NightMare/Desktop/neurofeed_back/course_data_clean.csv"

In [81]:
# Используйте метод read_csv() из библиотеки Pandas для чтения CSV файла
df = pd.read_csv(path_to_datasets, index_col=0)

In [82]:
df

,course_title,num_ratings,useful,easy,liked,num_reviews,reviews,course_rating,course_rating_int
course_code,,,,,,,,,
CS 115,Introduction to Computer Science 1,2111,21%,10%,23%,86,go to office hours and practice,liked course,1.0
CS 115,Introduction to Computer Science 1,2111,21%,10%,23%,86,One of my least favourite courses. Although th...,disliked course,0.0
CS 115,Introduction to Computer Science 1,2111,21%,10%,23%,86,It starts with a very low pace but after midte...,disliked course,0.0
CS 115,Introduction to Computer Science 1,2111,21%,10%,23%,86,Took this in 2018 with no programming experien...,liked course,1.0
CS 115,Introduction to Computer Science 1,2111,21%,10%,23%,86,I loved everything about cs 115. Great instruc...,liked course,1.0
...,...,...,...,...,...,...,...,...,...
ASL 102R,American Sign Language 2,1,100%,75%,100%,1,Took this online with Georgia Whalen. As it wa...,liked course,1.0
CS 136L,Tools and Techniques for Software Development,1,100%,100%,100%,1,I either designed or helped design the majorit...,liked course,1.0
AFM 417,Special Topics in Accounting,0,NaN,NaN,NaN,1,Topic 3: Intro to Data Analytics,NaN,NaN


In [83]:
# Сброс индексации
df.reset_index(drop=True, inplace=True)

In [84]:
df

,course_title,num_ratings,useful,easy,liked,num_reviews,reviews,course_rating,course_rating_int
0,Introduction to Computer Science 1,2111,21%,10%,23%,86,go to office hours and practice,liked course,1.0
1,Introduction to Computer Science 1,2111,21%,10%,23%,86,One of my least favourite courses. Although th...,disliked course,0.0
2,Introduction to Computer Science 1,2111,21%,10%,23%,86,It starts with a very low pace but after midte...,disliked course,0.0
3,Introduction to Computer Science 1,2111,21%,10%,23%,86,Took this in 2018 with no programming experien...,liked course,1.0
4,Introduction to Computer Science 1,2111,21%,10%,23%,86,I loved everything about cs 115. Great instruc...,liked course,1.0
...,...,...,...,...,...,...,...,...,...
14833,American Sign Language 2,1,100%,75%,100%,1,Took this online with Georgia Whalen. As it wa...,liked course,1.0
14834,Tools and Techniques for Software Development,1,100%,100%,100%,1,I either designed or helped design the majorit...,liked course,1.0
14835,Special Topics in Accounting,0,NaN,NaN,NaN,1,Topic 3: Intro to Data Analytics,NaN,NaN
14836,Senior Honours Essay,0,100%,0%,NaN,1,Not sure if I liked this course yet. Definitel...,NaN,NaN


In [85]:
# Переименование столбца "labels" в "label"
df.rename(columns={"reviews": "text"}, inplace=True)
# Удаляем столбец 'Numbers'
df = df.drop(['course_title', 'num_ratings', 'useful', 'easy', 'liked', 'num_reviews', 'course_rating'], axis=1)

In [86]:
df

,text,course_rating_int
0,go to office hours and practice,1.0
1,One of my least favourite courses. Although th...,0.0
2,It starts with a very low pace but after midte...,0.0
3,Took this in 2018 with no programming experien...,1.0
4,I loved everything about cs 115. Great instruc...,1.0
...,...,...
14833,Took this online with Georgia Whalen. As it wa...,1.0
14834,I either designed or helped design the majorit...,1.0
14835,Topic 3: Intro to Data Analytics,NaN
14836,Not sure if I liked this course yet. Definitel...,NaN


In [87]:
# Переименование столбца "labels" в "label"
df.rename(columns={"course_rating_int": "label"}, inplace=True)

In [88]:
df

,text,label
0,go to office hours and practice,1.0
1,One of my least favourite courses. Although th...,0.0
2,It starts with a very low pace but after midte...,0.0
3,Took this in 2018 with no programming experien...,1.0
4,I loved everything about cs 115. Great instruc...,1.0
...,...,...
14833,Took this online with Georgia Whalen. As it wa...,1.0
14834,I either designed or helped design the majorit...,1.0
14835,Topic 3: Intro to Data Analytics,NaN
14836,Not sure if I liked this course yet. Definitel...,NaN


In [89]:
# Удаление строк с пропущенными значениями (NaN) в столбце "label"
df = df.dropna(subset=['label'])

# Преобразование значений столбца "label" из float в целочисленный формат
df['label'] = df['label'].astype(int)

C:\Temp\ipykernel_13952\2794349432.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['label'].astype(int)


In [90]:
df

,text,label
0,go to office hours and practice,1
1,One of my least favourite courses. Although th...,0
2,It starts with a very low pace but after midte...,0
3,Took this in 2018 with no programming experien...,1
4,I loved everything about cs 115. Great instruc...,1
...,...,...
14830,This is a really wonderful course! Dr. Fraser ...,1
14831,While this course ended up being relatively in...,1
14832,Personally did not find Computational Biomecha...,0
14833,Took this online with Georgia Whalen. As it wa...,1


In [91]:
import re

# Функция для удаления знаков препинания из текста
def remove_punctuation(text):
    if isinstance(text, str):
        return re.sub(r'[^\w\s]', '', text)
    else:
        return None

# Удаление знаков препинания из текста
df['text'] = df['text'].apply(remove_punctuation)

# Удаление строк с пропущенными значениями в столбце "text"
df = df.dropna(subset=['text'])

# Приведение текста к нижнему регистру
df['text'] = df['text'].str.lower()

C:\Temp\ipykernel_13952\3538449472.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(remove_punctuation)
C:\Temp\ipykernel_13952\3538449472.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.lower()


In [92]:
df

,text,label
0,go to office hours and practice,1
1,one of my least favourite courses although thi...,0
2,it starts with a very low pace but after midte...,0
3,took this in 2018 with no programming experien...,1
4,i loved everything about cs 115 great instruct...,1
...,...,...
14830,this is a really wonderful course dr fraser ea...,1
14831,while this course ended up being relatively in...,1
14832,personally did not find computational biomecha...,0
14833,took this online with georgia whalen as it was...,1


In [93]:
# Функция для перевода текста
def translate_text(text):
    return argos_en_ru.translate(text)

# Применение функции перевода с использованием потоков
with ThreadPoolExecutor() as executor:
    tqdm.pandas()
    df['text'] = list(tqdm(executor.map(translate_text, df['text']), total=len(df)))

100%|██████████| 14608/14608 [3:52:28<00:00,  1.05it/s]  
C:\Temp\ipykernel_13952\4076156546.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = list(tqdm(executor.map(translate_text, df['text']), total=len(df)))


In [94]:
df.to_csv("course_data_clean_ru.csv", index=False)

In [95]:
df

,text,label
0,ходить в офис и практиковаться,1
1,"Один из моих наименее любимых курсов, хотя все...",0
2,"Он начинается с очень низкого темпа, но после ...",0
3,Я взял это в 2018 году без опыта программирова...,1
4,Я любил все о cs 115 великих инструкторов чест...,1
...,...,...
14830,"Это действительно замечательный курс, доктор Ф...",1
14831,Хотя этот курс оказался относительно интересны...,1
14832,Лично не нашел вычислительную биомеханику поле...,0
14833,"взял это онлайн с Джорджией китовой, как это б...",1
